From https://github.com/abhijeet3922/Object-recognition-CIFAR-10/blob/master/cifar10_90.py


In [ ]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np

def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    elif epoch > 100:
        lrate = 0.0003        
    return lrate

(x_train_original, y_train_original), (x_test_original, y_test_original) = cifar10.load_data()
x_train = x_train_original.astype('float32')
x_test = x_test_original.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

num_classes = 10
y_train = np_utils.to_categorical(y_train_original,num_classes)
y_test = np_utils.to_categorical(y_test_original,num_classes)

weight_decay = 1e-4
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.summary()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [97]:
#data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )

In [ ]:
history = datagen.fit(x_train)
#training
batch_size = 64

opt_rms = keras.optimizers.RMSprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),\
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=50,\
                    verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule)])

In [ ]:
from matplotlib import pyplot
pyplot.plot(history.history['accuracy'])
pyplot.plot(history.history['val_accuracy'])
pyplot.title('model accuracy')
pyplot.ylabel('accuracy')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'val'], loc='upper left')
pyplot.show()

In [ ]:
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['val_loss'])
pyplot.title('model loss')
pyplot.ylabel('loss')
pyplot.xlabel('epoch')
pyplot.legend(['train', 'val'], loc='upper left')
pyplot.show()

In [102]:
#save to disk
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)

model.save_weights('model.h5')    

#testing
scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

model.save('sampledata/keras/bigmodel')

79/79 [==============================] - 1s 7ms/step - loss: 0.6168 - accuracy: 0.8517

Test result: 85.170 loss: 0.617
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: sampledata/keras/bigmodel/assets


In [114]:
import shutil
shutil.make_archive('sampledata/keras/bigmodel', 'zip', 'sampledata/keras/bigmodel')
#shutil.unpack_archive('sampledata/keras/bigmodel.zip', extract_dir='sampledata/model')

#loadedmodel = keras.models.load_model('sampledata/keras/bigmodel')
#loadedscores = loadedmodel.evaluate(x_test, y_test, batch_size=128, verbose=1)
#print('\nTest result: %.3f loss: %.3f' % (loadedscores[1]*100,loadedscores[0]))

In [ ]:
from matplotlib import pyplot
from PIL.Image import fromarray
import numpy as np

print(x_test.shape)
print(y_test.shape)

for i in range(0, 9):
  pyplot.subplot(330 + 1 + i)
  pyplot.imshow(fromarray(x_test_original[i]))
  label = y_test_original[i][0]
  pyplot.title(label=names[label])

pyplot.show()

# Print some summaries about the expected and predicted values.
predictions = model.predict_classes(x_test)
predictednames = [names[i] for i in [x for x in predictions]]
uniquepredictions, predictioncounts = np.unique(predictions, return_counts=True)
print(predictednames)
print(uniquepredictions)
print(predictioncounts)
expectednames = [names[i] for i in [x[0] for x in y_test_original]]
uniqueexpected, uniquecounts = np.unique(y_test_original, return_counts = True)
print(expectednames)
print(uniqueexpected)
print(uniquecounts)

# Make a confusion matrix to show how well the classifier did
import pandas as pd
act = pd.Series(expectednames, name='Actual')
pre = pd.Series(predictednames, name='Predicted')
df_confusion = pd.crosstab(act, pre)
print(df_confusion)

In [116]:
# This cell reads some sample data
import os.path
covidfileexists = os.path.isfile('sampledata/keras/puss.png') 
if covidfileexists:
  pass
else:
  # a bit of magic to fetch the file
  !rm -rf sampledata
  !git clone https://github.com/awchisholm/sampledata.git

In [ ]:
from PIL import Image
im = Image.open("sampledata/keras/ship.jpg")
imrgb = im.convert('RGB')
size = 32, 32
im32 = imrgb.resize(size)
im32array = np.array(im32)
im32array_scaled = im32array.astype('float32')/255.0
test_array = im32array_scaled.reshape(1,32,32,3)

names = {0: 'aeroplane', 1: 'car', 2:'bird', 3:'cat', 4:'deer', 5:'dog', 6:'frog', 7:'horse', 8:'ship', 9:'lorry'}
# Now we can predict what it is
myprediction = model.predict_classes(test_array)
print([names[i] for i in [x for x in myprediction]])
pyplot.imshow(im)
#pyplot.imshow(im32)
myprediction[0]